# E. coli spike-in coverage

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import gzip
import tqdm.notebook as tqdm
import time

In [2]:
DISABLE_PBAR = False

In [3]:
SEQDIR = "/Users/addisonhowe/mnts/raw_data/sequencing"
DATDIRBASE = "/Users/addisonhowe/mnts/raw_data/sequencing/samples"

sam_header_fname = "header.sam"

outdir = f"../out/ecoli_coverage/arrays"
imgdir = f"../out/ecoli_coverage/images"
os.makedirs(outdir, exist_ok=True)
os.makedirs(imgdir, exist_ok=True)

In [4]:
# Need to fix this name since in the basecov file there is a comma.
FIX_NAMES = {
    "NC_010943.1_Stenotrophomonas_maltophilia_K279a__strain_K279a":
        "NC_010943.1_Stenotrophomonas_maltophilia_K279a,_strain_K279a",
}


In [5]:
RAWNAME_TO_SAMPLE = {}
SAMPLE_TO_RAWNAME = {}
with open(f"{SEQDIR}/raw_data_name_mapping.tsv", "r") as f:
    csvreader = csv.reader(f, delimiter="\t")
    for row in csvreader:  # process each row
        RAWNAME_TO_SAMPLE[row[0]] = row[1]
        SAMPLE_TO_RAWNAME[row[1]] = row[0]


In [6]:
REGION_MAPPING = {}  # Map name to gene to interval

for d in os.listdir("../igv/out/search_results"):
    search_file = f"../igv/out/search_results/{d}"
    with open(search_file, "r") as f:
        for line in f:
            # Skip comment lines (optional)
            if line.startswith("#") or not line.strip():
                continue
            parts = line.strip().split("\t")
            if len(parts) >= 5:
                seqname = parts[0]
                start = int(parts[3])
                end = int(parts[4])
                if seqname not in REGION_MAPPING:
                    REGION_MAPPING[seqname] = {}
                if d not in REGION_MAPPING[seqname]:
                    REGION_MAPPING[seqname][d] = []
                REGION_MAPPING[seqname][d].append((start, end))

for k in REGION_MAPPING:
    print(k, REGION_MAPPING[k])

NC_016830.1_Pseudomonas_fluorescens_F113 {'gene_narJ': [(4471254, 4472009), (4471254, 4472009)], 'gene_norW': [(4047309, 4048442), (4047309, 4048442)], 'gene_norV': [(4045870, 4047312), (4045870, 4047312)], 'gene_nosR': [(3532052, 3534220), (3532052, 3534220), (4024580, 4026706), (4024580, 4026706)], 'gene_nosZ': [(3530080, 3531966), (3530080, 3531966), (4026772, 4028709), (4026772, 4028709)], 'gene_napA': [(3593108, 3595612), (3593108, 3595612)], 'gene_narI': [(4472002, 4472682), (4472002, 4472682)], 'gene_nirD': [(2065052, 2065369), (2065052, 2065369), (3330304, 3330690), (3330304, 3330690)], 'gene_norR': [(4044188, 4045711), (4044188, 4045711), (4451523, 4453055), (4451523, 4453055), (5626742, 5628298), (5626742, 5628298)], 'gene_nirB': [(2065366, 2067819), (2065366, 2067819), (3327743, 3330304), (3327743, 3330304)], 'gene_narH': [(4469713, 4471251), (4469713, 4471251)], 'gene_napE': [(3595907, 3596074), (3595907, 3596074)]}
NZ_KB906274.1 {'gene_narJ': [(27471, 28184), (27471, 28184

In [7]:
array_list = [f for f in os.listdir(outdir) if f.endswith(".npz")]
key = "Lee_A8Q_1_Ecoli_contig_1_polypolish"

In [8]:
def find_regions(x, dx=1):
    regions = []
    r0 = x[0]
    r1 = np.nan
    idx_prev = x[0]
    for i in range(1, len(x)):
        idx = x[i]
        # Check if the current value is an extension of the current region
        if idx - idx_prev <= dx:
            # Extend the region
            r1 = idx
        else:
            # Reached end of regions. Store and reset.
            regions.append([r0, r1])
            r0 = idx
            r1 = np.nan
        idx_prev = idx
    # Append final region
    regions.append([r0, r1])
    return np.array(regions)

def get_spike_height(x, regions):
    spike_heights = np.zeros(len(regions))
    for i, region in enumerate(regions):
        r0, r1 = region
        if np.isnan(r1):
            r1 = r0
        heights = x[int(r0):int(r1 + 1)]
        spike_heights[i] = np.max(heights)
    return spike_heights

In [9]:
kappa = 5  # Track regions exceeding kappa * median reads
dx = 100  # Padding for regions used in `find_regions`

for array in tqdm.tqdm(array_list, total=len(array_list), disable=DISABLE_PBAR):
    x = np.load(f"{outdir}/{array}", allow_pickle=True)[key]

    med = np.median(x)  # store the median value
    spike_locations = np.argwhere(x > kappa * med).flatten()
    spike_regions = find_regions(spike_locations, dx=dx)
    spike_heights = get_spike_height(x, spike_regions)
    np.save(
        f"{outdir}/{array.replace(".npz", "")}_spike_regions.npy", 
        spike_regions
    )
    np.save(
        f"{outdir}/{array.replace(".npz", "")}_spike_heights.npy", 
        spike_heights
    )


    fig, ax = plt.subplots(1, 1, figsize=(8,6))
    # Plot base coverage
    ax.plot(np.arange(1, len(x) + 1), x, '.')
    
    # Add annotations
    annotations = REGION_MAPPING.get(key, {})
    ylims = ax.get_ylim()
    labelheight = ylims[1] - 0.025 * (ylims[1] - ylims[0])
    for gene in annotations:
        prev_start, prev_end = -1, -1
        regions = annotations[gene]
        for region in regions:
            start, end = region
            if prev_start != start or prev_end != end:
                ax.axvline(start, *ylims, linestyle=":", color="k", zorder=1)
                ax.axvline(end, *ylims, linestyle=":", color="k", zorder=1)
                tbox = ax.text(start, labelheight, gene.split("_")[1])
                labelheight -= 0.025 * (ylims[1] - ylims[0])
            prev_start, prev_end = start, end
    
    ax.set_xlabel(f"position")
    ax.set_ylabel("read count")
    title = key
    subtitle = array.removeprefix("coverage_arrays_").removesuffix(".npz")
    ax.set_title(title + "\n" + subtitle)

    plt.savefig(f"{imgdir}/{array.replace(".npz", "")}.png")
    plt.close()


  0%|          | 0/30 [00:00<?, ?it/s]

In [10]:
bin_frac = 0.001
for array in tqdm.tqdm(array_list, total=len(array_list), disable=DISABLE_PBAR):
    x = np.load(f"{outdir}/{array}", allow_pickle=True)[key]

    fig, ax = plt.subplots(1, 1, figsize=(8,6))
    n = len(x)
    binsize = int(np.ceil(bin_frac * n))
    nbins = n // binsize + (n % binsize != 0)
    remainder = n % binsize
    if remainder > 0:
        x = np.concatenate([x, np.zeros(binsize - remainder)])
    xmaxed = x.reshape([-1, binsize]).max(axis=1)
    # Plot base coverage
    ax.plot(np.arange(1, len(xmaxed) + 1), xmaxed, '.')
    
    # Add annotations
    annotations = REGION_MAPPING.get(key, {})
    ylims = ax.get_ylim()
    labelheight = ylims[1] - 0.025 * (ylims[1] - ylims[0])
    for gene in annotations:
        prev_start, prev_end = -1, -1
        regions = annotations[gene]
        for region in regions:
            start, end = region
            start = 1 + start // binsize
            end = 1 + end // binsize
            if prev_start != start or prev_end != end:
                ax.axvline(start, *ylims, linestyle=":", color="k", zorder=1)
                ax.axvline(end, *ylims, linestyle=":", color="k", zorder=1)
                tbox = ax.text(start, labelheight, gene.split("_")[1])
                labelheight -= 0.025 * (ylims[1] - ylims[0])
            prev_start, prev_end = start, end
    
    ax.set_xlabel(f"bin (size {binsize})")
    ax.set_ylabel("read count")
    title = key
    subtitle = array.removeprefix("coverage_arrays_").removesuffix(".npz")
    ax.set_title(title + "\n" + subtitle)

    plt.savefig(f"{imgdir}/{array.replace(".npz", "")}_binned.png")
    plt.close()


  0%|          | 0/30 [00:00<?, ?it/s]